In [1]:
!pip install pymongo neo4j pandas tabulate

import sqlite3
import json
from pymongo import MongoClient
from neo4j import GraphDatabase
import pandas as pd
from tabulate import tabulate

print("✓ Import thành công!")
print("=" * 80)
print("DEMO 3 PHƯƠNG THỨC TRIỂN KHAI ACCESS CONTROL TRÊN NOSQL")
print("=" * 80)

✓ Import thành công!
DEMO 3 PHƯƠNG THỨC TRIỂN KHAI ACCESS CONTROL TRÊN NOSQL


#### PHƯƠNG PHÁP 1 - MODIFY FORMAT
Nhúng access_control trực tiếp vào document
Database: SQLite với JSON

In [2]:
"""
🔧 CELL 2: PHƯƠNG THỨC 1 - MODIFY FORMAT
Nhúng access_control trực tiếp vào document
Database: SQLite với JSON
"""

print("\n" + "="*80)
print("PHƯƠNG THỨC 1: MODIFY FORMAT")
print("="*80)

# Kết nối SQLite
conn1 = sqlite3.connect(':memory:')  # In-memory database
cursor1 = conn1.cursor()

# Tạo bảng employees với access_control nhúng trong document
cursor1.execute('''
CREATE TABLE employees (
    id INTEGER PRIMARY KEY,
    document TEXT NOT NULL
)
''')

# Insert dữ liệu mẫu
employees_data = [
    {
        "employee_id": 1,
        "name": "Alice",
        "salary": 50000,
        "department": "Engineering",
        "access_control": {
            "readable_by": ["HR_Manager", "Department_Manager", "Self"],
            "writable_by": ["HR_Manager", "Self"]
        }
    },
    {
        "employee_id": 2,
        "name": "Bob",
        "salary": 80000,
        "department": "Engineering",
        "access_control": {
            "readable_by": ["HR_Manager", "Department_Manager"],
            "writable_by": ["HR_Manager"]
        }
    },
    {
        "employee_id": 3,
        "name": "Charlie",
        "salary": 45000,
        "department": "HR",
        "access_control": {
            "readable_by": ["HR_Manager", "Self"],
            "writable_by": ["HR_Manager", "Self"]
        }
    },
    {
        "employee_id": 4,
        "name": "Diana",
        "salary": 90000,
        "department": "Engineering",
        "access_control": {
            "readable_by": ["HR_Manager"],
            "writable_by": ["HR_Manager"]
        }
    },
    {
        "employee_id": 5,
        "name": "Eve",
        "salary": 35000,
        "department": "Marketing",
        "access_control": {
            "readable_by": ["HR_Manager", "Department_Manager", "Self"],
            "writable_by": ["Self"]
        }
    }
]

for emp in employees_data:
    cursor1.execute('INSERT INTO employees (document) VALUES (?)', 
                   (json.dumps(emp),))

conn1.commit()

print("✓ Đã tạo database và insert 5 employees")
print("\n📝 Cấu trúc dữ liệu:")
print("   Mỗi employee là 1 document JSON với access_control nhúng bên trong")
print("\nVí dụ 1 document:")
print(json.dumps(employees_data[0], indent=2, ensure_ascii=False))



PHƯƠNG THỨC 1: MODIFY FORMAT
✓ Đã tạo database và insert 5 employees

📝 Cấu trúc dữ liệu:
   Mỗi employee là 1 document JSON với access_control nhúng bên trong

Ví dụ 1 document:
{
  "employee_id": 1,
  "name": "Alice",
  "salary": 50000,
  "department": "Engineering",
  "access_control": {
    "readable_by": [
      "HR_Manager",
      "Department_Manager",
      "Self"
    ],
    "writable_by": [
      "HR_Manager",
      "Self"
    ]
  }
}


In [3]:
"""
🔍 CELL 3: DEMO QUERIES - MODIFY FORMAT
Thực hiện các query kiểm tra quyền ở nhiều levels
"""

print("\n" + "="*80)
print("DEMO QUERIES - MODIFY FORMAT")
print("="*80)

# Query 1: User-level - Alice có thể đọc thông tin của chính mình không?
print("\n📌 Query 1: User-level - Alice đọc thông tin của chính mình")
query1 = """
SELECT json_extract(document, '$.name') as name,
       json_extract(document, '$.salary') as salary,
       json_extract(document, '$.access_control.readable_by') as readable_by
FROM employees
WHERE json_extract(document, '$.employee_id') = 1
  AND json_extract(document, '$.access_control.readable_by') LIKE '%Self%'
"""
result1 = cursor1.execute(query1).fetchall()
print(f"SQL: {query1}")
print(f"Kết quả: {result1}")
print("✓ Alice CÓ QUYỀN đọc thông tin của chính mình")

# Query 2: Role-level - HR_Manager có thể đọc thông tin của ai?
print("\n📌 Query 2: Role-level - HR_Manager xem được bao nhiêu employees?")
query2 = """
SELECT json_extract(document, '$.employee_id') as id,
       json_extract(document, '$.name') as name,
       json_extract(document, '$.salary') as salary
FROM employees
WHERE json_extract(document, '$.access_control.readable_by') LIKE '%HR_Manager%'
"""
result2 = cursor1.execute(query2).fetchall()
print(f"SQL: {query2}")
print(f"\nKết quả: HR_Manager có thể đọc {len(result2)} employees:")
for row in result2:
    print(f"  - ID: {row[0]}, Name: {row[1]}, Salary: {row[2]}")

# Query 3: Department-level - Employees nào trong Engineering có thể được Department_Manager đọc?
print("\n📌 Query 3: Department-level - Department_Manager xem Engineering dept")
query3 = """
SELECT json_extract(document, '$.employee_id') as id,
       json_extract(document, '$.name') as name,
       json_extract(document, '$.department') as dept
FROM employees
WHERE json_extract(document, '$.department') = 'Engineering'
  AND json_extract(document, '$.access_control.readable_by') LIKE '%Department_Manager%'
"""
result3 = cursor1.execute(query3).fetchall()
print(f"SQL: {query3}")
print(f"\nKết quả: {len(result3)} employees trong Engineering:")
for row in result3:
    print(f"  - ID: {row[0]}, Name: {row[1]}, Dept: {row[2]}")

# Query 4: Write permission - Ai có thể WRITE (sửa) thông tin của Bob?
print("\n📌 Query 4: Write permission - Ai có thể sửa thông tin Bob?")
query4 = """
SELECT json_extract(document, '$.name') as name,
       json_extract(document, '$.access_control.writable_by') as writable_by
FROM employees
WHERE json_extract(document, '$.employee_id') = 2
"""
result4 = cursor1.execute(query4).fetchone()
print(f"SQL: {query4}")
print(f"Kết quả: {result4[1]}")
print("✓ Chỉ có HR_Manager có quyền WRITE")

# Query 5: Conditional - Employees có salary <= 50000 và HR_Manager có thể đọc
print("\n📌 Query 5: Conditional - Employees có salary <= 50k mà HR có thể đọc")
query5 = """
SELECT json_extract(document, '$.name') as name,
       json_extract(document, '$.salary') as salary
FROM employees
WHERE CAST(json_extract(document, '$.salary') AS INTEGER) <= 50000
  AND json_extract(document, '$.access_control.readable_by') LIKE '%HR_Manager%'
"""
result5 = cursor1.execute(query5).fetchall()
print(f"SQL: {query5}")
print(f"\nKết quả: {len(result5)} employees thỏa điều kiện:")
for row in result5:
    print(f"  - Name: {row[0]}, Salary: ${row[1]}")

print("\n" + "="*80)
print("ƯU NHƯỢC ĐIỂM - MODIFY FORMAT")
print("="*80)
print("\n✅ ƯU ĐIỂM:")
print("  1. Đơn giản - Mỗi document tự quản lý quyền")
print("  2. Không cần JOIN - AC nằm ngay trong document")
print("  3. Nhanh với index - O(1) nếu có index trên employee_id")
print("  4. Dễ hiểu - Developer dễ debug và maintain")

print("\n❌ NHƯỢC ĐIỂM:")
print("  1. Redundancy CAO - Mỗi document chứa AC riêng")
print("  2. Khó cập nhật policy - Phải update TẤT CẢ documents")
print("  3. Tốn storage - AC được lặp lại nhiều lần")
print("  4. Khó audit - Phải scan toàn bộ để biết ai có quyền gì")
print("  5. JSON query không hiệu quả - LIKE '%text%' chậm")



DEMO QUERIES - MODIFY FORMAT

📌 Query 1: User-level - Alice đọc thông tin của chính mình
SQL: 
SELECT json_extract(document, '$.name') as name,
       json_extract(document, '$.salary') as salary,
       json_extract(document, '$.access_control.readable_by') as readable_by
FROM employees
WHERE json_extract(document, '$.employee_id') = 1
  AND json_extract(document, '$.access_control.readable_by') LIKE '%Self%'

Kết quả: [('Alice', 50000, '["HR_Manager","Department_Manager","Self"]')]
✓ Alice CÓ QUYỀN đọc thông tin của chính mình

📌 Query 2: Role-level - HR_Manager xem được bao nhiêu employees?
SQL: 
SELECT json_extract(document, '$.employee_id') as id,
       json_extract(document, '$.name') as name,
       json_extract(document, '$.salary') as salary
FROM employees
WHERE json_extract(document, '$.access_control.readable_by') LIKE '%HR_Manager%'


Kết quả: HR_Manager có thể đọc 5 employees:
  - ID: 1, Name: Alice, Salary: 50000
  - ID: 2, Name: Bob, Salary: 80000
  - ID: 3, Name: Char

#### PHƯƠNG PHÁP 2 - HIERARCHICAL INTEGRATION
Tách biệt data và AC rules, tích hợp qua query logic
Database: MongoDB (hoặc in-memory dict nếu không có MongoDB)

In [ ]:
print("\n" + "="*80)
print("PHƯƠNG THỨC 2: HIERARCHICAL INTEGRATION")
print("="*80)

# Sử dụng dictionary để mô phỏng MongoDB (không cần cài MongoDB server)
# Trong thực tế: client = MongoClient('mongodb://localhost:27017/')
class MockMongoDB:
    def __init__(self):
        self.db = {
            'departments': [],
            'employees': []
        }
    
    def insert_department(self, dept):
        self.db['departments'].append(dept)
    
    def insert_employee(self, emp):
        self.db['employees'].append(emp)
    
    def find_department(self, query):
        for dept in self.db['departments']:
            match = all(dept.get(k) == v for k, v in query.items())
            if match:
                return dept
        return None
    
    def find_employees(self, query):
        results = []
        for emp in self.db['employees']:
            match = all(emp.get(k) == v for k, v in query.items())
            if match:
                results.append(emp)
        return results

db2 = MockMongoDB()

# Tạo departments với access_rules tập trung
departments = [
    {
        "department_id": "dept_engineering",
        "name": "Engineering",
        "access_rules": {
            "manager": {
                "query": {},  # No restrictions
                "allowed_actions": ["read", "write"]
            },
            "employee": {
                "query": {"salary": {"$lte": 60000}},  # Chỉ xem salary <= 60k
                "allowed_actions": ["read"]
            },
            "hr": {
                "query": {},
                "allowed_actions": ["read", "write", "delete"]
            }
        }
    },
    {
        "department_id": "dept_hr",
        "name": "HR",
        "access_rules": {
            "manager": {
                "query": {},
                "allowed_actions": ["read", "write"]
            },
            "employee": {
                "query": {"salary": {"$lte": 50000}},
                "allowed_actions": ["read"]
            },
            "hr": {
                "query": {},
                "allowed_actions": ["read", "write", "delete"]
            }
        }
    }
]

for dept in departments:
    db2.insert_department(dept)

# Insert employees (không có AC trong document!)
employees = [
    {"employee_id": 1, "name": "Alice", "salary": 50000, "department": "dept_engineering"},
    {"employee_id": 2, "name": "Bob", "salary": 80000, "department": "dept_engineering"},
    {"employee_id": 3, "name": "Charlie", "salary": 45000, "department": "dept_hr"},
    {"employee_id": 4, "name": "Diana", "salary": 90000, "department": "dept_engineering"},
    {"employee_id": 5, "name": "Eve", "salary": 35000, "department": "dept_hr"}
]

for emp in employees:
    db2.insert_employee(emp)

print("✓ Đã tạo database với 2 departments và 5 employees")
print("\n📝 Cấu trúc dữ liệu:")
print("   1. Departments collection: Chứa access_rules tập trung")
print("   2. Employees collection: Chỉ chứa data, KHÔNG có AC")
print("\nVí dụ Department với access_rules:")
print(json.dumps(departments[0], indent=2, ensure_ascii=False))
print("\nVí dụ Employee (KHÔNG có access_control):")
print(json.dumps(employees[0], indent=2, ensure_ascii=False))



PHƯƠNG THỨC 2: HIERARCHICAL INTEGRATION
✓ Đã tạo database với 2 departments và 5 employees

📝 Cấu trúc dữ liệu:
   1. Departments collection: Chứa access_rules tập trung
   2. Employees collection: Chỉ chứa data, KHÔNG có AC

Ví dụ Department với access_rules:
{
  "department_id": "dept_engineering",
  "name": "Engineering",
  "access_rules": {
    "manager": {
      "query": {},
      "allowed_actions": [
        "read",
        "write"
      ]
    },
    "employee": {
      "query": {
        "salary": {
          "$lte": 60000
        }
      },
      "allowed_actions": [
        "read"
      ]
    },
    "hr": {
      "query": {},
      "allowed_actions": [
        "read",
        "write",
        "delete"
      ]
    }
  }
}

Ví dụ Employee (KHÔNG có access_control):
{
  "employee_id": 1,
  "name": "Alice",
  "salary": 50000,
  "department": "dept_engineering"
}


In [19]:
"""
🔍 CELL 5: DEMO QUERIES - HIERARCHICAL INTEGRATION
Query với logic kiểm tra rules từ department
"""

print("\n" + "="*80)
print("DEMO QUERIES - HIERARCHICAL INTEGRATION")
print("="*80)

def check_access_hierarchical(user_role, dept_id, employee_id, action):
    """Helper function để check access theo hierarchical rules"""
    # Lấy department và rules
    dept = db2.find_department({"department_id": dept_id})
    if not dept:
        return False, "Department not found"
    
    rules = dept['access_rules'].get(user_role, {})
    if action not in rules.get('allowed_actions', []):
        return False, f"Action '{action}' not allowed for role '{user_role}'"
    
    # Lấy employee
    emps = db2.find_employees({"employee_id": employee_id, "department": dept_id})
    if not emps:
        return False, "Employee not found"
    
    emp = emps[0]
    
    # Check query conditions
    query_conditions = rules.get('query', {})
    for field, condition in query_conditions.items():
        if field in emp:
            if '$lte' in condition:
                if emp[field] > condition['$lte']:
                    return False, f"Condition failed: {field} > {condition['$lte']}"
            if '$gte' in condition:
                if emp[field] < condition['$gte']:
                    return False, f"Condition failed: {field} < {condition['$gte']}"
    
    return True, emp

# Query 1: User-level - Employee role xem Alice (salary 50k)
print("\n📌 Query 1: User-level - Employee role xem Alice (salary $50k)")
print("Logic: Lấy rules từ department → Check conditions → Return result")
allowed, result = check_access_hierarchical('employee', 'dept_engineering', 1, 'read')
print(f"User role: employee")
print(f"Target: Alice (salary $50,000)")
print(f"Department rule: employee chỉ xem salary <= $60,000")
print(f"Kết quả: {'✓ ALLOWED' if allowed else '✗ DENIED'}")
if allowed:
    print(f"Data: {result}")

# Query 2: User-level - Employee role xem Bob (salary 80k)
print("\n📌 Query 2: User-level - Employee role xem Bob (salary $80k)")
allowed, result = check_access_hierarchical('employee', 'dept_engineering', 2, 'read')
print(f"User role: employee")
print(f"Target: Bob (salary $80,000)")
print(f"Department rule: employee chỉ xem salary <= $60,000")
print(f"Kết quả: {'✓ ALLOWED' if allowed else '✗ DENIED - ' + result}")

# Query 3: Role-level - Manager xem tất cả trong Engineering
print("\n📌 Query 3: Role-level - Manager xem tất cả Engineering employees")
dept = db2.find_department({"department_id": "dept_engineering"})
rules = dept['access_rules']['manager']
print(f"Manager rules: {rules}")

emps = db2.find_employees({"department": "dept_engineering"})
accessible = []
for emp in emps:
    allowed, _ = check_access_hierarchical('manager', 'dept_engineering', emp['employee_id'], 'read')
    if allowed:
        accessible.append(emp)

print(f"Kết quả: Manager có thể xem {len(accessible)}/{len(emps)} employees:")
for emp in accessible:
    print(f"  - {emp['name']}: ${emp['salary']}")

# Query 4: Department-level - HR role xem tất cả departments
print("\n📌 Query 4: Department-level - HR role xem mọi department")
all_emps = db2.db['employees']
hr_accessible = []

for emp in all_emps:
    allowed, result = check_access_hierarchical('hr', emp['department'], emp['employee_id'], 'read')
    if allowed:
        hr_accessible.append(result)

print(f"HR rules: Không có query restrictions")
print(f"Kết quả: HR có thể xem {len(hr_accessible)} employees:")
for emp in hr_accessible:
    print(f"  - {emp['name']} ({emp['department']}): ${emp['salary']}")

# Query 5: Action-level - Employee có quyền WRITE không?
print("\n📌 Query 5: Action-level - Employee role có quyền WRITE không?")
allowed, result = check_access_hierarchical('employee', 'dept_engineering', 1, 'write')
print(f"User role: employee")
print(f"Action: write")
dept = db2.find_department({"department_id": "dept_engineering"})
print(f"Allowed actions for employee: {dept['access_rules']['employee']['allowed_actions']}")
print(f"Kết quả: {'✓ ALLOWED' if allowed else '✗ DENIED - ' + result}")

print("\n" + "="*80)
print("ƯU NHƯỢC ĐIỂM - HIERARCHICAL INTEGRATION")
print("="*80)
print("\n✅ ƯU ĐIỂM:")
print("  1. Tách biệt rõ ràng - Data và AC logic độc lập")
print("  2. Tiết kiệm storage - Rules tập trung ở department level")
print("  3. Dễ thay đổi policy - Chỉ sửa rules, không động vào data")
print("  4. Consistent - Mọi user trong dept dùng chung rules")
print("  5. Scalable - Thêm department mới dễ dàng")

print("\n❌ NHƯỢC ĐIỂM:")
print("  1. Query phức tạp hơn - Phải JOIN department + employees")
print("  2. Cần thêm logic - Application phải implement rule checking")
print("  3. Performance overhead - Mỗi query phải evaluate conditions")
print("  4. Khó debug - Logic nằm ở application layer, không phải DB")
print("  5. Không linh hoạt cho exceptions - Khó handle trường hợp đặc biệt")




DEMO QUERIES - HIERARCHICAL INTEGRATION

📌 Query 1: User-level - Employee role xem Alice (salary $50k)
Logic: Lấy rules từ department → Check conditions → Return result
User role: employee
Target: Alice (salary $50,000)
Department rule: employee chỉ xem salary <= $60,000
Kết quả: ✓ ALLOWED
Data: {'employee_id': 1, 'name': 'Alice', 'salary': 50000, 'department': 'dept_engineering'}

📌 Query 2: User-level - Employee role xem Bob (salary $80k)
User role: employee
Target: Bob (salary $80,000)
Department rule: employee chỉ xem salary <= $60,000
Kết quả: ✗ DENIED - Condition failed: salary > 60000

📌 Query 3: Role-level - Manager xem tất cả Engineering employees
Manager rules: {'query': {}, 'allowed_actions': ['read', 'write']}
Kết quả: Manager có thể xem 3/3 employees:
  - Alice: $50000
  - Bob: $80000
  - Diana: $90000

📌 Query 4: Department-level - HR role xem mọi department
HR rules: Không có query restrictions
Kết quả: HR có thể xem 5 employees:
  - Alice (dept_engineering): $50000
  -

#### PHƯƠNG PHÁP 3 - GRAPH MODEL
Biểu diễn bằng nodes và edges (Neo4j-style in-memory)

In [ ]:
print("\n" + "="*80)
print("PHƯƠNG THỨC 3: GRAPH MODEL")
print("="*80)

# Mock Graph DB (giống Neo4j)
class MockGraphDB:
    def __init__(self):
        self.nodes = {}
        self.edges = []
    
    def create_node(self, node_id, label, properties):
        self.nodes[node_id] = {"label": label, "properties": properties}
    
    def create_edge(self, from_id, to_id, rel_type, properties=None):
        self.edges.append({
            "from": from_id,
            "to": to_id,
            "type": rel_type,
            "properties": properties or {}
        })
    
    def match(self, pattern):
        """Simple pattern matching (simplified MATCH)"""
        results = []
        for edge in self.edges:
            if all(getattr(edge, k, edge.get(k)) == v for k, v in pattern.items()):
                results.append(edge)
        return results

graph = MockGraphDB()

# Tạo nodes: Users
graph.create_node("user_alice", "User", {"name": "Alice", "employee_id": 1})
graph.create_node("user_bob", "User", {"name": "Bob", "employee_id": 2})
graph.create_node("user_charlie", "User", {"name": "Charlie", "employee_id": 3})

# Tạo nodes: Roles
graph.create_node("role_manager", "Role", {"name": "manager", "level": 3})
graph.create_node("role_employee", "Role", {"name": "employee", "level": 1})
graph.create_node("role_hr", "Role", {"name": "hr_manager", "level": 5})

# Tạo nodes: Resources (Employees/Documents)
graph.create_node("resource_alice_profile", "Resource", {
    "name": "Alice's Profile", 
    "owner": "Alice",
    "sensitivity": 2
})
graph.create_node("resource_bob_profile", "Resource", {
    "name": "Bob's Profile",
    "owner": "Bob", 
    "sensitivity": 3
})
graph.create_node("resource_salary_report", "Resource", {
    "name": "Salary Report",
    "sensitivity": 5
})

# Relationships: User -[HAS_ROLE]-> Role
graph.create_edge("user_alice", "role_employee", "HAS_ROLE")
graph.create_edge("user_bob", "role_manager", "HAS_ROLE")
graph.create_edge("user_charlie", "role_hr", "HAS_ROLE")

# Relationships: Role -[CAN_READ]-> Resource (RBAC)
graph.create_edge("role_employee", "resource_alice_profile", "CAN_READ", {"level": 2})
graph.create_edge("role_manager", "resource_alice_profile", "CAN_READ", {"level": 3})
graph.create_edge("role_manager", "resource_bob_profile", "CAN_READ", {"level": 3})
graph.create_edge("role_hr", "resource_alice_profile", "CAN_READ", {"level": 5})
graph.create_edge("role_hr", "resource_bob_profile", "CAN_READ", {"level": 5})
graph.create_edge("role_hr", "resource_salary_report", "CAN_READ", {"level": 5})

# Relationships: User -[CAN_READ]-> Resource (Direct permission - PBAC)
graph.create_edge("user_alice", "resource_alice_profile", "CAN_READ", {"reason": "self"})

print("✓ Đã tạo graph database")
print(f"  - Nodes: {len(graph.nodes)} (Users, Roles, Resources)")
print(f"  - Edges: {len(graph.edges)} (HAS_ROLE, CAN_READ)")

print("\n📝 Cấu trúc Graph:")
print("\nNodes:")
for node_id, node in list(graph.nodes.items())[:5]:
    print(f"  ({node['label']}) {node_id}: {node['properties']}")

print("\nEdges (first 5):")
for edge in graph.edges[:5]:
    print(f"  ({edge['from']})-[{edge['type']}]->({edge['to']}) {edge['properties']}")



PHƯƠNG THỨC 3: GRAPH MODEL
✓ Đã tạo graph database
  - Nodes: 9 (Users, Roles, Resources)
  - Edges: 10 (HAS_ROLE, CAN_READ)

📝 Cấu trúc Graph:

Nodes:
  (User) user_alice: {'name': 'Alice', 'employee_id': 1}
  (User) user_bob: {'name': 'Bob', 'employee_id': 2}
  (User) user_charlie: {'name': 'Charlie', 'employee_id': 3}
  (Role) role_manager: {'name': 'manager', 'level': 3}
  (Role) role_employee: {'name': 'employee', 'level': 1}

Edges (first 5):
  (user_alice)-[HAS_ROLE]->(role_employee) {}
  (user_bob)-[HAS_ROLE]->(role_manager) {}
  (user_charlie)-[HAS_ROLE]->(role_hr) {}
  (role_employee)-[CAN_READ]->(resource_alice_profile) {'level': 2}
  (role_manager)-[CAN_READ]->(resource_alice_profile) {'level': 3}


In [23]:
"""
🔍 CELL 7: DEMO QUERIES - GRAPH MODEL
Query bằng graph traversal (MATCH pattern)
"""

print("\n" + "="*80)
print("DEMO QUERIES - GRAPH MODEL")
print("="*80)

def check_access_graph(user_id, resource_id, action):
    """Check access qua graph traversal"""
    # Method 1: Direct permission (PBAC - Figure 10)
    for edge in graph.edges:
        if (edge['from'] == user_id and 
            edge['to'] == resource_id and 
            edge['type'] == f'CAN_{action.upper()}'):
            return True, "Direct permission", edge
    
    # Method 2: Via role (RBAC - Figure 9)
    user_roles = [e['to'] for e in graph.edges 
                 if e['from'] == user_id and e['type'] == 'HAS_ROLE']
    
    resource_sensitivity = graph.nodes.get(resource_id, {}).get('properties', {}).get('sensitivity', 0)
    
    for role_id in user_roles:
        for edge in graph.edges:
            if (edge['from'] == role_id and 
                edge['to'] == resource_id and 
                edge['type'] == f'CAN_{action.upper()}'):
                
                permission_level = edge['properties'].get('level', 0)
                if permission_level >= resource_sensitivity:
                    return True, f"Via role: {role_id}", edge
                else:
                    return False, f"Insufficient level: {permission_level} < {resource_sensitivity}", None
    
    return False, "No permission found", None

# Query 1: PBAC (Figure 10) - Alice đọc profile của chính mình (Direct)
print("\n📌 Query 1: PBAC - Alice đọc profile của chính mình (Direct edge)")
print("Pattern: (user_alice)-[CAN_READ]->(resource_alice_profile)")
allowed, reason, edge = check_access_graph("user_alice", "resource_alice_profile", "read")
print(f"Kết quả: {'✓ ALLOWED' if allowed else '✗ DENIED'}")
print(f"Reason: {reason}")
if edge:
    print(f"Edge: {edge}")

# Query 2: RBAC (Figure 9) - Bob (manager) đọc Alice's profile (Via role)
print("\n📌 Query 2: RBAC - Bob (manager) đọc Alice's profile (Via role)")
print("Pattern: (user_bob)-[HAS_ROLE]->(role_manager)-[CAN_READ]->(resource_alice_profile)")
allowed, reason, edge = check_access_graph("user_bob", "resource_alice_profile", "read")
print(f"Kết quả: {'✓ ALLOWED' if allowed else '✗ DENIED'}")
print(f"Reason: {reason}")
if edge:
    print(f"Permission level: {edge['properties'].get('level')}")
    print(f"Resource sensitivity: {graph.nodes['resource_alice_profile']['properties']['sensitivity']}")

# Query 3: RBAC với level check - Employee xem Bob's profile
print("\n📌 Query 3: RBAC với level check - Alice (employee) xem Bob's profile")
print("Pattern: (user_alice)-[HAS_ROLE]->(role_employee)-[CAN_READ?]->(resource_bob_profile)")
allowed, reason, edge = check_access_graph("user_alice", "resource_bob_profile", "read")
print(f"Kết quả: {'✓ ALLOWED' if allowed else '✗ DENIED'}")
print(f"Reason: {reason}")
print(f"Employee level: {graph.nodes['role_employee']['properties']['level']}")
print(f"Bob's profile sensitivity: {graph.nodes['resource_bob_profile']['properties']['sensitivity']}")

# Query 4: Role-level - Charlie (HR) xem Salary Report
print("\n📌 Query 4: Role-level - Charlie (HR) xem Salary Report")
print("Pattern: (user_charlie)-[HAS_ROLE]->(role_hr)-[CAN_READ]->(resource_salary_report)")
allowed, reason, edge = check_access_graph("user_charlie", "resource_salary_report", "read")
print(f"Kết quả: {'✓ ALLOWED' if allowed else '✗ DENIED'}")
print(f"Reason: {reason}")
if edge:
    print(f"HR level: {graph.nodes['role_hr']['properties']['level']}")
    print(f"Salary report sensitivity: {graph.nodes['resource_salary_report']['properties']['sensitivity']}")



DEMO QUERIES - GRAPH MODEL

📌 Query 1: PBAC - Alice đọc profile của chính mình (Direct edge)
Pattern: (user_alice)-[CAN_READ]->(resource_alice_profile)
Kết quả: ✓ ALLOWED
Reason: Direct permission
Edge: {'from': 'user_alice', 'to': 'resource_alice_profile', 'type': 'CAN_READ', 'properties': {'reason': 'self'}}

📌 Query 2: RBAC - Bob (manager) đọc Alice's profile (Via role)
Pattern: (user_bob)-[HAS_ROLE]->(role_manager)-[CAN_READ]->(resource_alice_profile)
Kết quả: ✓ ALLOWED
Reason: Via role: role_manager
Permission level: 3
Resource sensitivity: 2

📌 Query 3: RBAC với level check - Alice (employee) xem Bob's profile
Pattern: (user_alice)-[HAS_ROLE]->(role_employee)-[CAN_READ?]->(resource_bob_profile)
Kết quả: ✗ DENIED
Reason: No permission found
Employee level: 1
Bob's profile sensitivity: 3

📌 Query 4: Role-level - Charlie (HR) xem Salary Report
Pattern: (user_charlie)-[HAS_ROLE]->(role_hr)-[CAN_READ]->(resource_salary_report)
Kết quả: ✓ ALLOWED
Reason: Via role: role_hr
HR level: 5


#### ĐÁNH GIÁ 3 CÁCH TRIỂN KHAI

In [29]:
"""
📊 CELL 10: VISUALIZE SO SÁNH
Biểu đồ radar chart so sánh 3 phương thức
"""

import numpy as np

print("\n" + "="*80)
print("BIỂU ĐỒ SO SÁNH")
print("="*80)

# Data for radar chart (scale 1-5, 5 is best)
categories = ['Performance', 'Flexibility', 'Ease of Use', 
              'Scalability', 'Storage Efficiency', 'Update Policy']

modify_format_scores = [5, 2, 5, 3, 2, 2]  # Nhanh, đơn giản, nhưng khó update
hierarchical_scores = [3, 3, 3, 5, 5, 5]   # Cân bằng, tốt cho enterprise
graph_scores = [2, 5, 2, 5, 3, 4]          # Linh hoạt nhưng phức tạp

# Number of variables
# num_vars = len(categories)

# # Compute angle for each axis
# angles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()
# angles += angles[:1]  # Complete the circle

# Data
modify_format_scores += modify_format_scores[:1]
hierarchical_scores += hierarchical_scores[:1]
graph_scores += graph_scores[:1]

# Plot
# fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(projection='polar'))

# Draw one axis per variable and add labels
# ax.set_theta_offset(np.pi / 2)
# ax.set_theta_direction(-1)
# ax.set_xticks(angles[:-1])
# ax.set_xticklabels(categories, size=12)

# # Draw ylabels
# ax.set_ylim(0, 5)
# ax.set_yticks([1, 2, 3, 4, 5])
# ax.set_yticklabels(['1', '2', '3', '4', '5'], size=10)
# ax.grid(True)

# # Plot data
# ax.plot(angles, modify_format_scores, 'o-', linewidth=2, 
#         label='Modify Format', color='#3498db')
# ax.fill(angles, modify_format_scores, alpha=0.15, color='#3498db')

# ax.plot(angles, hierarchical_scores, 'o-', linewidth=2, 
#         label='Hierarchical', color='#e74c3c')
# ax.fill(angles, hierarchical_scores, alpha=0.15, color='#e74c3c')

# ax.plot(angles, graph_scores, 'o-', linewidth=2, 
#         label='Graph Model', color='#2ecc71')
# ax.fill(angles, graph_scores, alpha=0.15, color='#2ecc71')

# Add legend
# ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1), fontsize=12)
# plt.title('So sánh 3 Phương thức Triển khai Access Control', 
#           size=15, weight='bold', pad=20)
# plt.tight_layout()
# plt.show()

print("\n📊 Giải thích điểm số (1-5, 5 là tốt nhất):")
print("\nModify Format:")
print("  Performance: 5/5 - Nhanh nhất với index")
print("  Flexibility: 2/5 - Khó thay đổi policy")
print("  Ease of Use: 5/5 - Đơn giản nhất")
print("  Scalability: 3/5 - OK với hệ thống vừa")
print("  Storage: 2/5 - Tốn storage nhất")
print("  Update Policy: 2/5 - Khó update")

print("\nHierarchical:")
print("  Performance: 3/5 - Trung bình (phải check conditions)")
print("  Flexibility: 3/5 - Cân bằng")
print("  Ease of Use: 3/5 - Cần hiểu query logic")
print("  Scalability: 5/5 - Rất tốt cho enterprise")
print("  Storage: 5/5 - Tiết kiệm nhất")
print("  Update Policy: 5/5 - Dễ nhất")

print("\nGraph Model:")
print("  Performance: 2/5 - Chậm với traversal phức tạp")
print("  Flexibility: 5/5 - Linh hoạt nhất")
print("  Ease of Use: 2/5 - Khó học Cypher")
print("  Scalability: 5/5 - Rất tốt")
print("  Storage: 3/5 - Nhiều edges")
print("  Update Policy: 4/5 - Dễ (add/remove edges)")




BIỂU ĐỒ SO SÁNH

📊 Giải thích điểm số (1-5, 5 là tốt nhất):

Modify Format:
  Performance: 5/5 - Nhanh nhất với index
  Flexibility: 2/5 - Khó thay đổi policy
  Ease of Use: 5/5 - Đơn giản nhất
  Scalability: 3/5 - OK với hệ thống vừa
  Storage: 2/5 - Tốn storage nhất
  Update Policy: 2/5 - Khó update

Hierarchical:
  Performance: 3/5 - Trung bình (phải check conditions)
  Flexibility: 3/5 - Cân bằng
  Ease of Use: 3/5 - Cần hiểu query logic
  Scalability: 5/5 - Rất tốt cho enterprise
  Storage: 5/5 - Tiết kiệm nhất
  Update Policy: 5/5 - Dễ nhất

Graph Model:
  Performance: 2/5 - Chậm với traversal phức tạp
  Flexibility: 5/5 - Linh hoạt nhất
  Ease of Use: 2/5 - Khó học Cypher
  Scalability: 5/5 - Rất tốt
  Storage: 3/5 - Nhiều edges
  Update Policy: 4/5 - Dễ (add/remove edges)
